<a href="https://colab.research.google.com/github/Deok97/AI_personal_study/blob/main/Overfeat_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Google Drive와 연결시 사용

In [ ]:
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# %cd /content/gdrive/MyDrive/Colab\ Notebooks

/content/gdrive/MyDrive/Colab Notebooks




---



In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, MaxPooling2D, Flatten
from tensorflow.keras import Model
from tensorflow.keras.initializers import RandomNormal 
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD
import tensorflow.keras.backend as K
from tensorflow.keras.losses import CategoricalCrossentropy
from keras.datasets import mnist
from mnist_local_generator import mnist_localization_generator
from utils import *
from google.colab import drive

### mnist_localization_generator 설명

In [2]:
"""
Signature: mnist_localization_generator(train_shape, test_shape, background=True, n_sample=None, image_size_range=None, image_ratio_range=None)

Description:
mnist 데이터를 온라인으로 다운받고 다운받은 mnist 데이터를 지정된 크기의 이미지 안에 random 으로 붙입니다.
그리고 붙여진 위치 정보를 반환합니다.
train 이미지와 test 이미지 모두 진행됩니다.
"""

mnist_localization_generator?

In [4]:
"""
Description
Description:
  mnist 데이터를 온라인으로 다운받고 다운받은 mnist 데이터를 지정된 크기의 이미지 안에 random 으로 붙입니다.
  그리고 붙여진 위치 정보를 반환합니다.
  train 이미지와 test 이미지 모두 진행됩니다.

: return
  train_images: ndarray, 4d array, shape = (N, H, W, C (단 C=1))
  train_cls_true: ndarray, 2d array, shape = (N, 1, 1, 11)
  train_reg_true: ndarray, 2d array, shape =(N, 1, 1 , 4), x1, y1, x2, y2  좌표 형태를 가짐
  test_images: ndarray, 4d array, shape = (N, H, W, C (단 C=1))
  test_cls_true: ndarray, 2d array, shape = (N, 1, 1, 11)
  test_reg_true: ndarray, 2d array, shape = (N, 1, 1, 4), x1, y1, x2, y2  좌표 형태를 가짐

"""

# 훈련 이미지, 시험 이미지는 (231, 231) size을 갖음
# background가 0.1*n_sample 개 만큼 추가된 데이터를 갖음
# background 데이터의 클래스 인덱스는 11
(train_imgs, train_cls, train_reg), (test_imgs, test_cls, test_reg) = \
mnist_localization_generator(train_shape=(231, 231),
                             test_shape=(231, 231),
                             background=True,
                             n_sample=1000)

배경 class가 포함되어 class 는 11로 설정 되어 있습니다.


100%|██████████| 1000/1000 [00:02<00:00, 345.56it/s]


object 가 없는 background 이미지를 추가합니다.(전체 이미지의 10%)
background 의 class 는 11로 설정 되어 있습니다.
background 의 regression 는 (0,0,0,0)로 설정 되어 있습니다.


100%|██████████| 100/100 [00:00<00:00, 354.04it/s]


In [5]:
train_imgs.shape, train_cls.shape, train_reg.shape

((1100, 231, 231, 1), (1100, 1, 1, 11), (1100, 1, 1, 4))

In [ ]:
K.clear_session()

# Overfeat 논문에서는 FC Layer을 Convolutional Layer로 변환
# Fully Convolutional Network을 구축
# input layer
input_ = Input(shape=(231, 231, 1))

# Backbone - 1st layer
conv_0 = Conv2D(filters=96, kernel_size=(11, 11), strides=4, activation='relu',
                kernel_initializer=RandomNormal(0, 0.01), kernel_regularizer=l2(0.00001))(input_)
max_0 = MaxPooling2D(pool_size=2, strides=2)(conv_0)

# Backbone - 2nd layer
conv_1 = Conv2D(filters=256, kernel_size = 5, strides=1, padding='valid', activation='relu',
                kernel_initializer=RandomNormal(0, 0.01), kernel_regularizer=l2(0.00001))(max_0)
max_1 = MaxPooling2D(pool_size=2, strides=2)(conv_1)

# Backbone - 3rd layer
conv_2 = Conv2D(filters = 512, kernel_size=3, strides = 1, padding='same', activation='relu',
                kernel_initializer=RandomNormal(0, 0.01), kernel_regularizer=l2(0.00001))(max_1)

# Backbone - 4th layer
conv_3 = Conv2D(filters = 1024, kernel_size=3, strides = 1, padding='same', activation='relu',
                kernel_initializer=RandomNormal(0, 0.01), kernel_regularizer=l2(0.00001))(conv_2)

# Backbone - 5th layer
conv_4 = Conv2D(filters=1024, kernel_size = 3, strides=1, padding='same', activation='relu',
                kernel_initializer=RandomNormal(0, 0.01), kernel_regularizer=l2(0.00001))(conv_3)
max_2 = MaxPooling2D(pool_size=2, strides=2)(conv_4)

# Clasisfication - 6th layer
conv_5 = Conv2D(filters=3072, kernel_size = 6, strides=1, padding='valid', activation='relu',
                kernel_initializer=RandomNormal(0, 0.01), kernel_regularizer=l2(0.00001))(max_2)

# Clasisfication - 7th layer
conv_6 = Conv2D(filters=4096, kernel_size = 1, strides=1, padding='same', activation='relu',
                kernel_initializer=RandomNormal(0, 0.01), kernel_regularizer=l2(0.00001))(conv_5)

# Clasisfication - 8th layer; output layer
output_ = Conv2D(filters=1000, kernel_size = 1, strides=1, padding='same', activation='softmax')(conv_6)

model = Model(input_, output_)
sgd = SGD(0.05, momentum=0.6)
model.compile(optimizer=sgd, loss=CategoricalCrossentropy, metrics=['cee'])

model.summary()

print(output_)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 231, 231, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 56, 56, 96)        11712     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 28, 28, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 256)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 512)       118016

In [ ]:
model.fit(train_imgs, train_cls)


TypeError: ignored

In [ ]:
model.predict()